# Sparse Hebbian Learning with full homeostasis : testing different rescaling functions

In this notebook, we test the convergence of SparseNet as a function of different parameters tuning the quantization. These parameters only influence the way we select dictionary elements and thus the homeostasis.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL_set
tag = 'rescaling'
#opts = dict(homeo_method='HEH', eta_homeo=0.05, C=10., nb_quant=256, P_cum=None, verbose=1)
#opts = dict(homeo_method='HEH', n_iter=513, record_each=32, eta_homeo=0.1, C=5., nb_quant=256, P_cum=None, verbose=1)
opts = dict(homeo_method='HEH', verbose=1)

experiments = SHL_set(opts, tag=tag)
list_figures = []
list_figures = ['show_dico', 'show_Pcum']
display_variables = ['error', 'logL', 'qerror', 'aerror', 'MI', 'cputime']
display_variables = ['error', 'MC', 'cputime']


Extracting data..loading the data called : data_cache/data_data
Data is of shape : (65520, 324) - done in 0.10s.


In [4]:
!ls -l {experiments.shl.data_cache}/{tag}*
!ls -l {experiments.shl.data_cache}/{tag}*lock*

-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 27 18:46 data_cache/rescaling - C=1.25000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 27 20:27 data_cache/rescaling - C=1.98425_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 28 02:32 data_cache/rescaling - C=12.59921_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 28 03:54 data_cache/rescaling - C=20.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 27 22:03 data_cache/rescaling - C=3.14980_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 27 23:40 data_cache/rescaling - C=5.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2405197 Apr 28 01:10 data_cache/rescaling - C=7.93701_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  1858300 Apr 28 06:30 data_cache/rescaling - nb_quant=101_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  5114646 Apr 28 13:03 data_cache/rescaling - nb_quant=1024_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2069980 Apr 28 07:47 data_cache/rescaling - 

## different rescaling values

In [ ]:
experiments.scan(variable='C', list_figures=list_figures, display='')
plt.show()
for display_variable in display_variables:
    fig, ax = experiments.scan(variable='C', list_figures=[], display='final', display_variable=display_variable)
    #plt.show()

No cache found data_cache/rescaling - C=0.50000_dico.pkl: Learning the dictionary with algo = mp 
 Training on 65520 patches
[dict_learning] Iteration   1 /   4097 (elapsed time:   1s,   0mn   1s)
Iteration  129 /   4097 (elapsed time:  150s,   2mn  30s)
Iteration  257 /   4097 (elapsed time:  298s,   4mn  58s)
Iteration  385 /   4097 (elapsed time:  467s,   7mn  47s)
Iteration  513 /   4097 (elapsed time:  630s,  10mn  30s)
Iteration  641 /   4097 (elapsed time:  787s,  13mn   7s)
Iteration  769 /   4097 (elapsed time:  940s,  15mn  40s)
Iteration  897 /   4097 (elapsed time:  1088s,  18mn   8s)
Iteration  1025 /   4097 (elapsed time:  1233s,  20mn  33s)
Iteration  1153 /   4097 (elapsed time:  1380s,  23mn   0s)
Iteration  1281 /   4097 (elapsed time:  1524s,  25mn  24s)
Iteration  1409 /   4097 (elapsed time:  1669s,  27mn  49s)
Iteration  1537 /   4097 (elapsed time:  1812s,  30mn  12s)
Iteration  1665 /   4097 (elapsed time:  1958s,  32mn  38s)
Iteration  1793 /   4097 (elapsed ti

## different quantization parameters

with a fixed $C$, we now change the number of quantization steps used to compute the histogram:

In [ ]:
experiments.scan(variable='nb_quant', list_figures=list_figures, display='')
plt.show()
for display_variable in display_variables:
    fig, ax = experiments.scan(variable='nb_quant', list_figures=[], display='final', display_variable=display_variable)
    #plt.show()

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts